In [7]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
plt.rc('font', family='D2Coding')
from plotnine import *
import requests

In [3]:
# 법정동의 면적 구하기

In [4]:
width = pd.read_csv('./data/행정동 면적.csv')
width

,행정동명,면적
0,가락1동,1.34
1,가락2동,0.96
2,가락본동,1.13
3,가리봉동,0.40
4,가산동,2.52
...,...,...
421,효창동,0.44
422,후암동,0.86
423,휘경1동,0.63
424,휘경2동,1.05


In [5]:
# 수치의 편의를 위해 면적 X 1000을 해준다.
width['면적'] = width['면적']  * 1000
width

,행정동명,면적
0,가락1동,1340.0
1,가락2동,960.0
2,가락본동,1130.0
3,가리봉동,400.0
4,가산동,2520.0
...,...,...
421,효창동,440.0
422,후암동,860.0
423,휘경1동,630.0
424,휘경2동,1050.0


네이버 지오코딩

In [8]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 칼럼을 추가합니다.
width['위도'] = None
width['경도'] = None

# 네이버 지오코딩 API를 호출하여 주소별 위도와 경도를 얻습니다.
for index, row in width.iterrows():
    address = row['행정동명']
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('addresses'):
            latitude = data['addresses'][0]['y']
            longitude = data['addresses'][0]['x']
            width.at[index, '위도'] = latitude
            width.at[index, '경도'] = longitude

# 결과를 출력합니다.
width

,행정동명,면적,위도,경도
0,가락1동,1340.0,37.4964898,127.1098653
1,가락2동,960.0,37.4987054,127.1266714
2,가락본동,1130.0,37.4955568,127.1217864
3,가리봉동,400.0,37.4825557,126.8897489
4,가산동,2520.0,37.4769094,126.8917326
...,...,...,...,...
421,효창동,440.0,37.5424569,126.9618498
422,후암동,860.0,37.5486375,126.9781344
423,휘경1동,630.0,37.592932,127.065725
424,휘경2동,1050.0,37.5902093,127.0685788


In [17]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
for index, row in width.iterrows():
    latitude = row['위도']
    longitude = row['경도']
#     https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=129.1133567,35.2982640&sourcecrs=epsg:4326&output=json&orders=addr,admcode
    if latitude is not None and longitude is not None:
        url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get('results'):
                gu = data['results'][0]['region']['area2']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                dong = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                width.at[index, '법정동'] = dong  # 주소 정보를 데이터프레임에 추가
                width.at[index, '시군구'] = gu  # 주소 정보를 데이터프레임에 추가
                
width

,행정동명,면적,위도,경도,법정동명,주소,법정동,시군구
0,가락1동,1340.0,37.4964898,127.1098653,None,가락동,가락동,송파구
1,가락2동,960.0,37.4987054,127.1266714,None,가락동,가락동,송파구
2,가락본동,1130.0,37.4955568,127.1217864,None,가락동,가락동,송파구
3,가리봉동,400.0,37.4825557,126.8897489,None,가리봉동,가리봉동,구로구
4,가산동,2520.0,37.4769094,126.8917326,None,가산동,가산동,금천구
...,...,...,...,...,...,...,...,...
421,효창동,440.0,37.5424569,126.9618498,None,효창동,효창동,용산구
422,후암동,860.0,37.5486375,126.9781344,None,후암동,후암동,용산구
423,휘경1동,630.0,37.592932,127.065725,None,휘경동,휘경동,동대문구
424,휘경2동,1050.0,37.5902093,127.0685788,None,휘경동,휘경동,동대문구


In [19]:
legal_dong = width.groupby(['시군구','법정동'])['면적'].sum().reset_index()
legal_dong

,시군구,법정동,면적
0,강남구,개포동,6510.0
1,강남구,논현동,2720.0
2,강남구,대치동,3520.0
3,강남구,도곡동,2040.0
4,강남구,삼성동,5840.0
...,...,...,...
211,중랑구,면목동,6010.0
212,중랑구,묵동,1870.0
213,중랑구,상봉동,1530.0
214,중랑구,신내동,3490.0


In [20]:
legal_dong.to_csv('./data/법정동면적.csv')

In [21]:
legal_dong_dongbuk = Legal_Dong[(Legal_Dong['시군구'] == '강북구') | (Legal_Dong['시군구'] == '도봉구') | (Legal_Dong['시군구'] == '노원구')| (Legal_Dong['시군구'] == '중랑구')
                            | (Legal_Dong['시군구'] == '성북구')| (Legal_Dong['시군구'] == '동대문구')]
legal_dong_dongbuk

,시군구,법정동,면적
21,강북구,미아동,4370.0
22,강북구,번동,2710.0
23,강북구,수유동,6240.0
24,강북구,우이동,10280.0
55,노원구,공릉동,8230.0
56,노원구,상계동,15680.0
57,노원구,월계동,4270.0
58,노원구,중계동,5200.0
59,노원구,하계동,2060.0
60,도봉구,도봉동,9550.0


In [23]:
legal_dong_dongbuk.to_csv('./data/동북부면적.csv')